In [39]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from multiprocessing import Pool
import time
from tqdm.notebook import tqdm
import missingno as msno
from utils import get_idle_proportion



In [40]:
"""" 
We would like to set up a ML problem. 

In this problem we would like to predict the proportion of being idle for the next interval for a given node.
This makes the problem a time series problem with regression falvour as we predict a continues value
between 0 and 1.  


Another setup would be to turn this into a classification problem by cutting the interval [0-1] into multiple classes

class low idle: [0-0.3]
class middle idle: [0.3-0.9]
class high idle: [0.9-1]

Then we predict the next step belongs to which class. This is more realistic because differenticating between 
0.91 and 0.92 does not have much value(right?)
"""

'" \nWe would like to set up a ML problem. \n\nIn this problem we would like to predict the proportion of being idle for the next interval for a given node.\nThis makes the problem a time series problem with regression falvour as we predict a continues value\nbetween 0 and 1.  \n\n\nAnother setup would be to turn this into a classification problem by cutting the interval [0-1] into multiple classes\n\nclass low idle: [0-0.3]\nclass middle idle: [0.3-0.9]\nclass high idle: [0.9-1]\n\nThen we predict the next step belongs to which class. This is more realistic because differenticating between \n0.91 and 0.92 does not have much value(right?)\n'

In [41]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.precision = 4 # show 4 digits precision
folder_path_slurm_data = Path('./system_analytics_2024/slurm_data')
folder_path_prom_data = Path('./system_analytics_2024/prom_data')
folder_path_saving_results = Path('./results')



In [36]:

# for getting the up-to-date data run the clenaing_sinfo file
# with the latest data from the system.
df = pd.read_parquet(folder_path_slurm_data / 'sinfo_EDA_cleaned.parquet.gzip')
df.sample(n=10)


,node,time,state,node_type,time_30min_interval,time_1hour_interval,time_3hour_interval,time_2hour_interval,time_6hour_interval,time_12hour_interval,time_day_interval
122,tcn428,2024-11-15 07:21:46,allocated,tcn,2024-11-15 07:00:00,2024-11-15 07:00:00,2024-11-15 06:00:00,2024-11-15 06:00:00,2024-11-15 06:00:00,2024-11-15 00:00:00,2024-11-15
61,tcn478,2024-11-06 12:27:51,mixed,tcn,2024-11-06 12:00:00,2024-11-06 12:00:00,2024-11-06 12:00:00,2024-11-06 12:00:00,2024-11-06 12:00:00,2024-11-06 12:00:00,2024-11-06
205,tcn995,2024-11-05 09:03:51,idle,tcn,2024-11-05 09:00:00,2024-11-05 09:00:00,2024-11-05 09:00:00,2024-11-05 08:00:00,2024-11-05 06:00:00,2024-11-05 00:00:00,2024-11-05
4,tcn288,2024-11-14 21:13:07,drained,tcn,2024-11-14 21:00:00,2024-11-14 21:00:00,2024-11-14 21:00:00,2024-11-14 20:00:00,2024-11-14 18:00:00,2024-11-14 12:00:00,2024-11-14
252,tcn1188,2024-11-05 01:16:46,allocated,tcn,2024-11-05 01:00:00,2024-11-05 01:00:00,2024-11-05 00:00:00,2024-11-05 00:00:00,2024-11-05 00:00:00,2024-11-05 00:00:00,2024-11-05
134,tcn980,2024-11-08 16:27:51,idle,tcn,2024-11-08 16:00:00,2024-11-08 16:00:00,2024-11-08 15:00:00,2024-11-08 16:00:00,2024-11-08 12:00:00,2024-11-08 12:00:00,2024-11-08
342,tcn415,2024-11-05 22:09:56,allocated,tcn,2024-11-05 22:00:00,2024-11-05 22:00:00,2024-11-05 21:00:00,2024-11-05 22:00:00,2024-11-05 18:00:00,2024-11-05 12:00:00,2024-11-05
39,gcn61,2024-11-07 21:00:25,mixed,gcn,2024-11-07 21:00:00,2024-11-07 21:00:00,2024-11-07 21:00:00,2024-11-07 20:00:00,2024-11-07 18:00:00,2024-11-07 12:00:00,2024-11-07
26,tcn35,2024-11-09 01:04:37,allocated,tcn,2024-11-09 01:00:00,2024-11-09 01:00:00,2024-11-09 00:00:00,2024-11-09 00:00:00,2024-11-09 00:00:00,2024-11-09 00:00:00,2024-11-09
291,tcn333,2024-11-10 14:46:41,allocated,tcn,2024-11-10 14:30:00,2024-11-10 14:00:00,2024-11-10 12:00:00,2024-11-10 14:00:00,2024-11-10 12:00:00,2024-11-10 12:00:00,2024-11-10


In [42]:
""" 
TEMPORRAY: At the end of the EDA notebook or notebooks save a data frame for ML.
"""
# the time interval that we put all the states in it.
map_time_col = {'time_1hour_interval':pd.Timedelta('1h'),
                'time_30min_interval':pd.Timedelta('30min'),
                'time_2hour_interval':pd.Timedelta('2h'),
                'time_3hour_interval':pd.Timedelta('3h'),
                'time_6hour_interval':pd.Timedelta('6h'),
                'time_12hour_interval':pd.Timedelta('12h'),
                }
""
# pick a time col: 
# WHICH TIME INTEVAL MAKES SENSE FOR ML?
time_col = 'time_1hour_interval'
df_stat, df_idle, df_total = get_idle_proportion(df, time_col)
# show a smaple
df_stat.sample(n=10)



,node,time_1hour_interval,idle_duration,all_state_durations_in_interval,idle_proportion
378951,tcn837,2024-11-15 16:00:00,0 days 00:57:47,0 days 00:57:47,1.0
178877,tcn189,2024-11-10 20:00:00,0 days 00:00:00,0 days 00:59:08,0.0
313597,tcn626,2024-11-12 00:00:00,0 days 00:00:00,0 days 00:58:04,0.0
97575,tcn1063,2024-11-05 00:00:00,0 days 00:57:32,0 days 00:57:32,1.0
1769,fcn104,2024-11-08 17:00:00,0 days 00:58:54,0 days 00:58:54,1.0
133602,tcn118,2024-11-07 05:00:00,0 days 00:00:00,0 days 00:59:08,0.0
151603,tcn1237,2024-11-13 08:00:00,0 days 00:59:20,0 days 00:59:20,1.0
312440,tcn622,2024-11-10 07:00:00,0 days 00:00:00,0 days 00:59:21,0.0
322109,tcn654,2024-11-06 12:00:00,0 days 00:00:00,0 days 00:59:08,0.0
65159,gcn59,2024-11-13 05:00:00,0 days 00:59:11,0 days 00:59:11,1.0


In [15]:
""" 
ENRICHING THE DATA SET:

Get the last state of the node in the interval. I think it helps the ML algorithm to know not only
the proportion of the time that idle happened but also the actual state of the node.

# Leaking should not happen here. We should not use the state in the future interval

"""




# note the trick here! this happens again in the future for Promethues data
df_last = df.groupby(['node', time_col], as_index=False)[['node', time_col, 'state', 'time']].tail(1).copy()

# merge it with the main
df_stat = pd.merge(df_stat, df_last[['node', time_col, 'state', 'time']],
                   how='left', on=['node', time_col]).copy()
# rename it
df_stat.rename(columns={'time':'time_for_last_state', 'state': 'last_state'}, inplace=True)
# show a sample
df_stat.sample(n=10)


,node,time_3hour_interval,idle_duration,all_state_durations_in_interval,idle_proportion,last_state,time_for_last_state
39946,tcn1166,2024-11-09 21:00:00,0 days 02:59:04,0 days 02:59:04,1.0000,idle,2024-11-09 23:59:27
105302,tcn716,2024-11-09 09:00:00,0 days 00:00:00,0 days 02:58:30,0.0000,allocated,2024-11-09 11:59:47
122919,tcn900,2024-11-07 18:00:00,0 days 02:58:52,0 days 02:58:52,1.0000,idle,2024-11-07 20:59:55
15975,gcn16,2024-11-12 18:00:00,0 days 00:00:00,0 days 02:59:12,0.0000,mixed,2024-11-12 20:59:44
43977,tcn1208,2024-11-08 12:00:00,0 days 02:59:11,0 days 02:59:11,1.0000,idle,2024-11-08 14:59:38
108102,tcn746,2024-11-04 15:00:00,0 days 00:00:00,0 days 00:56:08,0.0000,allocated,2024-11-04 17:59:51
53819,tcn174,2024-11-13 06:00:00,0 days 00:58:05,0 days 02:59:25,0.3237,idle,2024-11-13 08:59:45
54925,tcn186,2024-11-11 18:00:00,0 days 00:00:00,0 days 02:59:14,0.0000,allocated,2024-11-11 20:59:36
71248,tcn359,2024-11-09 15:00:00,0 days 00:00:00,0 days 02:59:11,0.0000,allocated,2024-11-09 17:59:15
5913,fcn52,2024-11-12 18:00:00,0 days 02:59:12,0 days 02:59:12,1.0000,idle,2024-11-12 20:59:44


In [ ]:
""" 
ENRICHING THE DATA SET:
To be done
"""
# get the data from EAR for node waiting time; this enrich the data set



In [16]:
""" 
ENRICHING THE DATA SET:

Bring Promethues data set and get the last step or steps of the mesurements ==> 
WE MUST BRING NEWER DATA AS WELL. 
Is it possible to write a for loops and read the promethues data sets
that we have and do the merging? 
"""
df = pd.read_parquet(folder_path_prom_data / 'metrics_stats_4_sorted.parquet.gzip')
df.head(n=10)

# how to check this df is soreted and does not include duplicates?

,node,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_receive_drop_total_min,node_network_receive_drop_total_max,node_network_receive_drop_total_mean,node_network_recei

In [17]:

# turn the time to pandas time
df['time'] = pd.to_datetime(df['timestamp'], unit='s')
# add the interval tag
df[time_col] = ((df['time'] )).dt.floor(freq=map_time_col[time_col])
# show  a sample
df.head(n=10)

,node,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_receive_drop_total_min,node_network_receive_drop_total_max,node_network_receive_drop_total_mean,node_network_recei

In [18]:
""" get the last time from promethues data base: 
what was the last measurements from Promethues for this node? """


# note the trick here!
df_last = df.groupby(['node', time_col], as_index=False).tail(1).copy()
df_last.head()


,node,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_receive_drop_total_min,node_network_receive_drop_total_max,node_network_receive_drop_total_mean,node_network_recei

In [ ]:
"""  
As we can see many of the rows are NAN, this means that for that specifc node there was no data in Prom.
We have to load other stored data and do the same thing. 
"""
# merge the data with the df_stat
df_stat = pd.merge(df_stat, df_last,
                   how='left', on=['node', time_col]).copy()
df_stat.rename(columns={'time':'time_for_last_prom_measurement'}, inplace=True)
df_stat.sample(n=10)
# df_stat[df_stat['node'].isin(['fcn113', 'fcn114'])].head()

,node,time_3hour_interval,idle_duration,all_state_durations_in_interval,idle_proportion,last_state,time_for_last_state,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_r

In [ ]:
""""  
It makes much more sense to save the data here and actually create a note book for machine learning,
since getting all these data from different sources is time consuming.
"""